# SR Lines Indicator

This code tries to detect support and resistance lines in the market and draw them on tradingview using pine script for traders to use.

The code is an experiment so that I can get a small hands on experience with trading data.

## Preparing Data

In this part we get the daily price information from Yahoo finance API. The only processing is to convert datetime indices to python date objects as they are easier to work with in next stages.

In [1]:
import datetime

import pandas as pd
import yfinance as yf

In [2]:
def get_price_df(symbol: str, start_date: datetime.date, end_date: datetime.date) -> pd.DataFrame:
    df = yf.download(
        symbol,
        start=start_date.strftime('%Y-%m-%d'),
        end=end_date.strftime('%Y-%m-%d'),
    )
    return df.reindex(df.index.date)

## Moving Average

To detect trends I'm using moving averages. For computing it, we are using a technical analysis library called `ta` for this.

In [3]:
import pandas as pd
from ta import trend

In [4]:
def get_moving_average(price_df: pd.DataFrame, window: int) -> pd.Series:
    return trend.sma_indicator(
        close=price_df['Close'],
        window=window,
        fillna=True,
    )

## Trend Detection

The algorithm used to detect trends is using moving averages. Whenever a moving average changes direction we assume the trend has finished.

In [18]:
import pydantic
from enum import StrEnum, auto

In [19]:
class TrendDirection(StrEnum):
    UP = auto()
    DOWN = auto()


class Trend(pydantic.BaseModel):
    start_date: datetime.date
    end_date: datetime.date
    direction: TrendDirection

In [20]:
def get_trends(moving_averages: pd.Series) -> list[Trend]:
    trends = []

    trend_start_date = None
    trend_direction = None

    prev_key = None
    for key, value in moving_averages.items():
        if prev_key is None:
            trend_start_date = key
            prev_key = key
            continue

        prev_value = moving_averages.loc[prev_key]
        if value > prev_value:
            if trend_direction is None:
                # The first trend.
                trend_direction = TrendDirection.UP
            elif trend_direction is TrendDirection.DOWN:
                # Change of trend direction.
                trends.append(
                    Trend(
                        start_date=trend_start_date,
                        end_date=prev_key,
                        direction=trend_direction,
                    )
                )

                trend_start_date = prev_key
                trend_direction = TrendDirection.UP
        elif value < prev_value:
            if trend_direction is None:
                # The first trend.
                trend_direction = TrendDirection.DOWN
            elif trend_direction is TrendDirection.UP:
                # Change of trend direction.
                trends.append(
                    Trend(
                        start_date=trend_start_date,
                        end_date=prev_key,
                        direction=trend_direction,
                    )
                )

                trend_start_date = prev_key
                trend_direction = TrendDirection.DOWN

        prev_key = key

    return trends

## SR Lines

Now that we have the trend ends, we can extract support and resistance lines from them. These lines have a price which is the price of the day the trend finished, and a weight. For now we are computing the weight based on the length of trend which has finished.

In [21]:
import pydantic

In [22]:
class SRLine(pydantic.BaseModel):
    price: float
    weight: float

In [23]:
def get_sr_lines(
        price_df: pd.DataFrame,
        trends: list[Trend],
        length_factor: float = 1.0,
        static_factor: float = 1.0,
) -> list[SRLine]:
    sr_lines = []
    for trend in trends:
        length_weight = length_factor * (trend.end_date - trend.start_date).days
        sr_lines.append(
            SRLine(
                price=price_df.loc[trend.end_date]['Close'],
                weight=static_factor * length_weight,
            )
        )

    return sr_lines

## SR Line Optimization

Not all support/resistance lines are needed. So in this stage we are going to merge lines that are too close to eachother. The merged line has a weight which is the total of weights and it's price is a weighted average of the previous prices. 

In [11]:
def compute_price_threshold(sr_lines: list[SRLine], percentage: float = 1.0) -> float:
    max_v = max([sr_line.price for sr_line in sr_lines])
    min_v = min([sr_line.price for sr_line in sr_lines])
    return percentage * (max_v - min_v) / 100.0

In [12]:
def merge_sr_lines(sr_lines: list[SRLine], threshold: float = 10) -> list[SRLine]:
    sorted_sr_lines = sorted(sr_lines, key=lambda x: x.price)

    while True:
        change_happened = False

        for i, sr_line1 in enumerate(sorted_sr_lines):
            if i == len(sorted_sr_lines):
                break

            for sr_line2 in sorted_sr_lines[i + 1:]:
                if abs(sr_line1.price - sr_line2.price) > threshold:
                    continue

                new_price = ((sr_line1.price * sr_line1.weight) + (sr_line2.price * sr_line2.weight)) / (
                            sr_line1.weight + sr_line2.weight)
                new_weight = sr_line1.weight + sr_line2.weight
                sorted_sr_lines.append(
                    SRLine(
                        price=new_price,
                        weight=new_weight,
                    ),
                )

                sorted_sr_lines.remove(sr_line1)
                sorted_sr_lines.remove(sr_line2)
                change_happened = True
                break

            if change_happened:
                break

        if not change_happened:
            break

    return sorted_sr_lines

## Tradingview Output

To make this code easier to use for traders, this stage converts the SR lines to a pine script that can be used in tradingview. Simply we are going to copy the output of this function to the 

In [24]:
def generate_palm_script(sr_lines: list[SRLine]) -> str:
    max_weight = max([sr_line.weight for sr_line in sr_lines])
    min_weight = min([sr_line.weight for sr_line in sr_lines])
    
    lines = [
        '//@version=5',
        'indicator("SR Lines", overlay = true)',
    ]

    for sr_line in sr_lines:
        red = 70
        green = 130
        blue = 240
        transparency = min (20, 100 - (((sr_line.weight - min_weight) / (max_weight - min_weight)) * 100))
        lines.append(
            f'hline({sr_line.price},linestyle=hline.style_solid, color=color.rgb({red}, {green}, {blue}, {transparency}), linewidth=1)'
        )
    
    return '\n'.join(lines)

## Main

This part glues all the previous stages together.

In [25]:
SYMBOL = '^GSPC'
START_DATE = datetime.date(2020, 1, 1)
END_DATE = datetime.datetime.today().date()
MA_WINDOW_SIZES = [7, 15, 30]
MERGE_PRICE_THRESHOLD_PERCENTAGE = 5

In [26]:
price_df = get_price_df(symbol=SYMBOL, start_date=START_DATE, end_date=END_DATE)

all_sr_lines = []
for window in MA_WINDOW_SIZES:
    moving_averages = get_moving_average(price_df=price_df, window=window)
    trends = get_trends(moving_averages=moving_averages)
    sr_lines = get_sr_lines(
        price_df=price_df,
        trends=trends,
    )
    all_sr_lines.extend(sr_lines)

len(all_sr_lines)

[*********************100%%**********************]  1 of 1 completed


320

In [16]:
price_threshold = compute_price_threshold(sr_lines=all_sr_lines, percentage=MERGE_PRICE_THRESHOLD_PERCENTAGE)
optimized_sr_lines = merge_sr_lines(sr_lines=all_sr_lines, threshold=price_threshold)
len(optimized_sr_lines)

10

In [17]:
print(generate_palm_script(sr_lines=optimized_sr_lines))

//@version=5
indicator("SR Lines", overlay = true)
hline(2460.915866427951,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(2817.0124423740144,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(3153.244238021526,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(3483.933837890625,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(4706.978080610796,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(3335.985647411193,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(4013.792654111404,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(3824.8398571918015,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 16.978922716627636), linewidth=1)
hline(4199.254559536638,linestyle=hline.style_solid, color=color.rgb(70, 130, 240, 20), linewidth=1)
hline(4472.65104078783

## Result

![sr_line_indicator_result](../docs/images/sr_lines_indicator_result.png)

## Future Works

- Implement other weighing mechanisms for SR lines.
    - e.g. weight multiplier based on the moving average duration.
- Filter SR lines with weights lower than a specific threshold.
- Find a way to define the merge threshold percentage instead algorithmically.
- Provide a testing mechanism to validate the results.
    - We should separate the data into train/validate/test.
    - The reward function can be the distance between each line and all the trend lines.
- It would be nice if we could somehow run the pine script directly on tradingview instead of copying it there.